In [15]:
import os
import sys
import json
from datetime import datetime
import subprocess

# Path to Master Functions and Keys
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..', '..'))
sys.path.append(root_dir)

import Birdeye.Basics.Master_Functions as Master_Functions

In [16]:
API_Key = Master_Functions.API_Key
wallet = Master_Functions.wallet
chain = Master_Functions.chain # Choose between: solana,ethereum,arbitrum,avalanche,bsc,optimism,polygon,base,zksync

# Generating Data

In [17]:
# Specify the path to the Python file you want to run
python_file_path = 'Dataset_Generator_New_Tokens.py'

# Run the Python file
subprocess.run(['python', python_file_path])

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}

--------------------------------

Running on: solana

--------------------------------

NEW TOKEN FILTERS:

Min Liquidity: 10K
Max Liquidity: 100K
Min Market Cap: 100K
Max Market Cap: 300K
Filtering New Tokens for: 0 days, 1 hours, 5 minutes back
Token Launch Liquidity Filter: 10K

--------------------------------
 
 
--------------------------------



/home/ubuntu/Birdeye_Dashboard/Birdeye/Projects/Indicators/Fib/Dataset_Generator_New_Tokens.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_market_data_df = pd.concat([df for df in market_data_list if not df.empty], ignore_index=True)


https://dexscreener.com/solana/9tQzk7yh3Q8Sd1PoqUqNZruHMQnS9DTHmVcJX7CUpump - Market Cap: 240K
https://dexscreener.com/solana/ASBv8NaULiMxbwLRXvqZc5b2H4iUqF9cUUiLpLzppump - Market Cap: 217K
https://dexscreener.com/solana/D2JKTF9oKkWSNFhBr1GfiHWrGTvJYN63w7U9grdopump - Market Cap: 191K
https://dexscreener.com/solana/H8eK75hXWrZRtUSuewapGQPbyHhVKHppErs468vcpump - Market Cap: 139K
https://dexscreener.com/solana/G8ZgBbTkQHEWFFesz246E4d4TpcwTuoXXafzebuipump - Market Cap: 125K
https://dexscreener.com/solana/zGSm7WWkUgV6NqrU47nC1iLheZsWaRMyFnzVKTUpump - Market Cap: 118K
https://dexscreener.com/solana/Va7yGwNguNXGWstcufHA6s6rEVts1rNVkcv47LepN4S - Market Cap: 104K

Token summary data saved to: Data/New_Token_Data/2024_10_21/Token_Summary/2024_Oct_21_0805PM/new_tokens_mc_added_filtered_0.1M-0.3M_0d_1h_5m.csv

Number of Tokens Filtered: 7

--------------------------------
 
All Token summary data saved to: Data/New_Token_Data/2024_10_21/Token_Summary/2024_Oct_21_0805PM/new_tokens_mc_added_filtered

CompletedProcess(args=['python', 'Dataset_Generator_New_Tokens.py'], returncode=0)

# Import Data

In [18]:
# Get the most recent folder
base_path = 'Data/New_Token_Data'
current_date = datetime.now().strftime('%Y_%m_%d')
date_folder = os.path.join(base_path, current_date)
ohlcv_folder = os.path.join(date_folder, 'OHLCV_Data')
ohlcv_datetime_folder = Master_Functions.get_most_recent_folder(ohlcv_folder)

# Print the name of the folder being used for import
print(f"Importing OHLCV data from folder: {ohlcv_datetime_folder}")

# Import the OHLCV data
imported_ohlcv_data = Master_Functions.import_ohlcv_data(ohlcv_datetime_folder)

# Print the structure of the imported OHLCV data dictionary
print("\nStructure of imported OHLCV data:")
ohlcv_structure = {token: {timeframe: df.shape for timeframe, df in timeframes.items()} for token, timeframes in imported_ohlcv_data.items()}
print(json.dumps(ohlcv_structure, indent=2))

# Example of how to access the data
print("\nExample of accessing the data:")
for token_address, timeframes in imported_ohlcv_data.items():
    print(f"Token: {token_address}")
    for timeframe, df in timeframes.items():
        print(f"  Timeframe: {timeframe}")
        print(f"  Data shape: {df.shape}")
        print(f"  First few rows:")
        print(df.head())
        print("\n")
    break  # Only print the first token as an example

# Example of calculating simple metrics
def calculate_metrics(df):
    if df.empty:
        return {
            'last_price': None,
            'daily_return': None,
            'volatility': None
        }
    return {
        'last_price': df['close'].iloc[-1],
        'daily_return': (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100 if len(df) > 1 else 0,
        'volatility': df['close'].pct_change().std() * 100 if len(df) > 1 else 0
    }

# Calculate metrics for each token and timeframe
metrics = {}
for token_address, timeframes in imported_ohlcv_data.items():
    metrics[token_address] = {}
    for timeframe, df in timeframes.items():
        metrics[token_address][timeframe] = calculate_metrics(df)

# Display metrics
print("\nCalculated metrics:")
for token_address, timeframes in metrics.items():
    print(f"Token: {token_address}")
    for timeframe, metric in timeframes.items():
        print(f"  Timeframe: {timeframe}")
        if metric['last_price'] is not None:
            print(f"    Last Price: {metric['last_price']:.4f}")
            print(f"    Daily Return: {metric['daily_return']:.2f}%")
            print(f"    Volatility: {metric['volatility']:.2f}%")
        else:
            print("    No data available")
    print("\n")

Importing OHLCV data from folder: Data/New_Token_Data/2024_10_21/OHLCV_Data/2024_Oct_21_0805PM

Structure of imported OHLCV data:
{
  "zGSm7WWkUgV6NqrU47nC1iLheZsWaRMyFnzVKTUpump": {
    "1m": [
      12,
      7
    ],
    "5m": [
      3,
      7
    ],
    "3m": [
      5,
      7
    ],
    "4H": [
      2,
      7
    ],
    "1H": [
      2,
      7
    ],
    "15m": [
      2,
      7
    ],
    "1D": [
      1,
      7
    ],
    "30m": [
      2,
      7
    ]
  },
  "H8eK75hXWrZRtUSuewapGQPbyHhVKHppErs468vcpump": {
    "1m": [
      61,
      7
    ],
    "5m": [
      13,
      7
    ],
    "3m": [
      21,
      7
    ],
    "4H": [
      2,
      7
    ],
    "1H": [
      2,
      7
    ],
    "15m": [
      5,
      7
    ],
    "1D": [
      1,
      7
    ],
    "30m": [
      3,
      7
    ]
  },
  "D2JKTF9oKkWSNFhBr1GfiHWrGTvJYN63w7U9grdopump": {
    "1m": [
      12,
      7
    ],
    "5m": [
      3,
      7
    ],
    "3m": [
      5,
      7
    ],
    "4H": [
